In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold

In [ ]:
TrainFeat = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
TrainTargetScored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
TestFeat = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
Submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
# ENCODING
TrainFeat['cp_type'].replace(['ctl_vehicle', 'trt_cp'], [0,1], inplace=True)
TrainFeat['cp_dose'].replace(['D1', 'D2'], [0,1], inplace=True)

TestFeat['cp_type'].replace(['ctl_vehicle', 'trt_cp'], [0,1], inplace=True)
TestFeat['cp_dose'].replace(['D1', 'D2'], [0,1], inplace=True)

X=TrainFeat.drop(["sig_id"], axis =1)
y=TrainTargetScored.drop(["sig_id"], axis =1)
X_TestFeat=TestFeat.drop(["sig_id"], axis =1)

In [ ]:
# X AND y CONCATENATION
df_concat = pd.concat([X, y], axis=1)
#  DETECTION OF TARGETS WITH SUM <=10
v=np.array(y.sum())
pos = []
x = 10 #The required element
for i in range(len(v)):
    if v[i] <= x:
        pos.append(i)
        
# OVER-SAMPLING FOR TARGETS WITH SUM <=10       
from sklearn.utils import resample
u=0
j=len(X.columns) #875 colonnes
for i in pos:
    not_MoA= df_concat[df_concat.iloc[:,i+j]==0]
    MoA= df_concat[df_concat.iloc[:,i+j]==1]
    u=u+1
    # upsample minority
    MoA_upsampled = resample(MoA,
                          replace=True, # sample with replacement
                          n_samples=200, # match number in majority class
                          random_state=0) # reproducible results
    # combine majority and upsampled minority
    if u==1: upsampled = pd.concat([not_MoA, MoA, MoA_upsampled])
    else: upsampled = pd.concat([upsampled , MoA_upsampled])

In [ ]:
# Separate input features and targets
X_up=upsampled[upsampled.columns[0:875]]
y_up=upsampled[upsampled.columns[875:1081]]       
from sklearn.utils import resample
u=0
j=len(X.columns) #875 colonnes
for i in pos:
    not_MoA= df_concat[df_concat.iloc[:,i+j]==0]
    MoA= df_concat[df_concat.iloc[:,i+j]==1]
    u=u+1
    # upsample minority
    MoA_upsampled = resample(MoA,
                          replace=True, # sample with replacement
                          n_samples=200, # match number in majority class
                          random_state=0) # reproducible results
    # combine majority and upsampled minority
    if u==1: upsampled = pd.concat([not_MoA, MoA, MoA_upsampled])
    else: upsampled = pd.concat([upsampled , MoA_upsampled])

# Separate input features and target
X_up=upsampled[upsampled.columns[0:875]]
y_up=upsampled[upsampled.columns[875:1081]]

# Number of additional samples
a=len(X_up)-len(X)
print (X.shape)
print (X_up.shape)

In [ ]:
def classif2 (base, X, y, X_TestFeat):
    losses_NN=[] 
    kf = KFold(n_splits=3,random_state=100, shuffle=True)
    np.random.seed(1010)    
    X2=X.to_numpy()
    y2=y.to_numpy()
    X_TestFeat2=X_TestFeat.to_numpy()
    for train_index, test_index in kf.split(X2):
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]         
        model = MultiOutputClassifier(base)
        train = model.fit(X_train, y_train)
        predicted=np.transpose(np.array(train.predict_proba(X_test))[:, :,-1])
        predictedTestFeat=np.transpose(np.array(train.predict_proba(X_TestFeat2))[:, :,-1])      
        loss=log_loss(np.ravel(y_test), np.ravel(predicted))      
        print('Loss: '+str(loss)) 
        losses_NN.append(loss)
    print('Average Loss: '+str(np.average(losses_NN))) 
    return train, predictedTestFeat

In [ ]:
#SCV
base = SVC (C=1e-3, random_state=0, kernel='linear', probability=True, decision_function_shape= 'ovo' )
SCV_CV = classif2 (base, X_up, y_up, X_TestFeat)

In [ ]:
predictedTestFeat = SCV_CV[1]
Pred_OverS_SVC = pd.DataFrame(predictedTestFeat, columns = Submission.columns[1:])
Pred_OverS_SVC.insert(0,'sig_id', Submission.loc[:,'sig_id'])
Pred_OverS_SVC.to_csv('submission.csv', index=False)